In [78]:
import itertools as it
import pickle

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import (
    LabelEncoder,
    OneHotEncoder,
    OrdinalEncoder,
    StandardScaler,
)
from sklearn.tree import DecisionTreeClassifier

In [79]:
def preprocess(df: pd.DataFrame, cols_ord=[], cats_ord=[], cols_nom=[], cols_num=[]):
    cols_ord_ = list(set(cols_ord) & set(df.columns))
    cats_ord_ = [cats_ord[cols_ord.index(col)] for col in cols_ord_]
    cols_nom_ = list(set(cols_nom) & set(df.columns))
    cols_num_ = list(set(cols_num) & set(df.columns))

    transformers = []
    if cols_ord_:
        transform = (OrdinalEncoder(categories=cats_ord_), cols_ord_)
        transformers.append(transform)
    if cols_nom_:
        transform = (OneHotEncoder(), cols_nom_)
        transformers.append(transform)
    if cols_num_:
        transform = (StandardScaler(), cols_num_)
        transformers.append(transform)

    if transformers:
        steps = [("transformer", make_column_transformer(*transformers))]
        return Pipeline(steps).fit_transform(df)
    else:
        return df.to_numpy()


def search(X, y, path=None, cache=True, n_jobs=-1):
    try:
        if path is not None and cache:
            with open(path, "rb") as file:
                results = pickle.load(file)
        else:
            raise FileNotFoundError()
    except FileNotFoundError:
        pipe = Pipeline([("estimator", LogisticRegression())])
        search = GridSearchCV(
            pipe,
            param_grid=[
                {"estimator": [LogisticRegression()], "estimator__penalty": "l1"},
                {
                    "estimator": [DecisionTreeClassifier()],
                    "estimator__max_depth": [10, 20, None],
                },
                {
                    "estimator": [RandomForestClassifier()],
                    "estimator__n_estimators": [100, 250, 1000],
                },
            ],
            n_jobs=n_jobs,
            scoring="f1_micro",
        )
        _ = search.fit(X, y)
        results = pd.DataFrame(search.cv_results_).sort_values(
            by="mean_test_score", ascending=False
        )
        with open(results, "wb") as file:
            pickle.dump(results, file)

    return results

### Carregamento dos dados

In [80]:
# Características
df_sociais = pd.read_csv("data/trabalho5_dados_sociais_4.csv")
df_modulo1 = pd.read_csv("data/trabalho5_dados_modulo1_4.csv")
df_modulo2 = pd.read_csv("data/trabalho5_dados_ateh_modulo2_4.csv")

### Análise exploratória

#### Relações entre características

In [81]:
# TODO

#### Relações entre características e rótulos

In [82]:
# TODO

### Pré-processamento

In [83]:
# Rótulos
y = LabelEncoder().fit(["Não", "Sim"]).transform(df_modulo2["aprovado"])
df_sociais = df_sociais.drop(["id", "aprovado"], axis=1)
df_modulo1 = df_modulo1.drop(["id", "aprovado"], axis=1)
df_modulo2 = df_modulo2.drop(["id", "aprovado"], axis=1)

In [84]:
# Ordens das categorias
sn_cat = ["Não", "Sim"]
sexo_cat = ["Feminino", "Masculino"]
escolaridade_cat = [
    "Ensino Médio Completo",
    "Ensino Superior Incompleto",
    "Ensino Superior Completo",
    "Pós-graduação",
]
materialdidatico_cat = ["Adequado", "Muito adequado"]
prazoatividades_cat = [
    "Pouquíssimo flexível",
    "Pouco flexível",
    "Flexível",
    "Muito flexível",
]
interacaopares_cat = ["Importante", "Muito importante"]
organizacaocurso_cat = ["Organizado", "Muito organizado"]
import_ajud_tutor_cat = ["Às vezes", "Sempre"]
autoavaliacao_cat = [
    "Não, não considero",
    "Sim, considero, porém, poderia estar me esforçando mais",
    "Sim, considero",
]
pp_cat = [
    "Discordo totalmente",
    "Discordo",
    "Nem discordo, nem concordo",
    "Concordo",
    "Concordo totalmente",
]

In [85]:
# Variáveis ordinais
cols_ord = [
    "escolaridade",
    "materialdidatico",
    "prazoatividades",
    "interacaopares",
    "import.ajud.tutor",
    "autoavaliacao.x",
] + [f"pp{n + 1:03}" for n in range(37)]
cats_ord = [
    escolaridade_cat,
    materialdidatico_cat,
    prazoatividades_cat,
    interacaopares_cat,
    import_ajud_tutor_cat,
    autoavaliacao_cat,
] + [pp_cat] * 37

# Variáveis nominais
cols_nom = list(set(df_sociais.select_dtypes(object).columns) - set(cols_ord))

# Variáveis numéricas
cols_num = ["idade", "tempodeservico"]

In [86]:
X_sociais = preprocess(df_sociais, cols_ord, cats_ord, cols_nom, cols_num)
X_modulo1 = preprocess(df_modulo1, cols_ord, cats_ord, cols_nom, cols_num)
X_modulo2 = preprocess(df_modulo2, cols_ord, cats_ord, cols_nom, cols_num)

X_sociais

array([[ 3.        ,  1.        ,  3.        , ...,  0.        ,
        -0.11217859,  0.65818824],
       [ 3.        ,  1.        ,  3.        , ...,  1.        ,
         0.41079851,  1.40819926],
       [ 3.        ,  1.        ,  3.        , ...,  0.        ,
         0.41079851,  0.33675495],
       ...,
       [ 3.        ,  1.        ,  3.        , ...,  0.        ,
         0.01856569, -1.27041153],
       [ 3.        ,  1.        ,  3.        , ...,  0.        ,
         0.28005423,  0.01532165],
       [ 3.        ,  1.        ,  3.        , ...,  0.        ,
         0.14930996,  0.87247711]])

### Classificação: Dados socioeconômicos

### Classificação: Dados socioeconômicos + primeiro módulo

### Classificação: Todos os dados